# LunarLander-v2



##Source
https://github.com/nikhilbarhate99/PPO-PyTorch/blob/master/PPO_continuous.py

(and the discrete: https://github.com/nikhilbarhate99/PPO-PyTorch/blob/master/PPO.py )

[source spec](https://github.com/nikhilbarhate99/PPO-PyTorch):

Python 3.6

PyTorch 1.0

NumPy 1.15.3

gym 0.10.8

Pillow 5.3.0

##description for the discrete prob
https://github.com/openai/gym/wiki/Leaderboard#lunarlander-v2
•Landing pad is always at coordinates (0,0).
•Coordinates are the first two numbers in state vector.
•Reward for moving from the top of the screen to landing pad and zero speed is about 100..140 points. If lander moves away from landing pad it loses reward back.
•Episode finishes if the lander crashes or comes to rest, receiving additional -100 or +100 points. Each leg ground contact is +10. Firing main engine is -0.3 points each frame. Solved is 200 points. Landing outside landing pad is possible. Fuel is infinite, so an agent can learn to fly and then land on its first attempt.
Four discrete actions available: do nothing, fire left orientation engine, fire main engine, fire right orientation engine.
LunarLander-v2 defines "solving" as getting average reward of 200 over 100 consecutive trials

##description for the continous prob
https://github.com/openai/gym/wiki/Leaderboard#lunarlandercontinuous-v2

•Landing pad is always at coordinates (0,0).
Coordinates are the first two numbers in state vector. Reward for moving from the top of the screen to landing pad and zero speed is about 100..140 points.
If lander moves away from landing pad it loses reward back.
Episode finishes if the lander crashes or comes to rest, receiving additional -100 or +100 points. Each leg ground contact is +10. Firing main engine is -0.3 points each frame. Solved is 200 points. Landing outside landing pad is possible. Fuel is infinite, so an agent can learn to fly and then land on its first attempt.
•Action is two real values vector from -1 to +1. First controls main engine, -1..0 off, 0..+1 throttle from 50% to 100% power. **Engine can't work with less than 50% power (j'espere qu il n a pas implementé qlq chose du genre)**.
• Second value -1.0..-0.5 fire left engine, +0.5..+1.0 fire right engine, -0.5..0.5 off.


#Code

In [37]:
import os

try:
  import torch
  import torch.nn as nn
  from torch.distributions import MultivariateNormal
  import gym
  import numpy as np
  from netsapi.challenge import *
except:
  !pip3 install git+https://github.com/slremy/netsapi --user --upgrade
  !pip install box2d-py
  print("restart the kernel and go")
  os._exit(0)  

# for the KDD-env
from sys import exit, exc_info, argv
from multiprocessing import Pool, current_process
import random as rand
import json
import requests
import numpy as np
import pandas as pd


import statistics
from IPython.display import clear_output
from contextlib import contextmanager
import sys, os
@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout
print("import successful")

import tensorflow as tf
import math
import matplotlib.pyplot as plt

import successful


In [0]:
device = "cpu"#torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Memory:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
    
    def clear_memory(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]


###nn.Linear           https://pytorch.org/docs/stable/_modules/torch/nn/modules/linear.html

Args:

        in_features: size of each input sample
        out_features: size of each output sample
        bias: If set to ``False``, the layer will not learn an additive bias.   Default: ``True``
        
### torch.distributions.multivariate_normal.MultivariateNormal
https://pytorch.org/docs/stable/distributions.html#multivariatenormal

Parameters

         loc (Tensor) – mean of the distribution         
         covariance_matrix (Tensor) – positive-definite covariance matrix
         precision_matrix (Tensor) – positive-definite precision matrix
         scale_tril (Tensor) – lower-triangular factor of covariance, with positive-valued diagonal
Ex
        m = MultivariateNormal(torch.zeros(2), torch.eye(2))
        m.sample()  # normally distributed with mean=`[0,0]` and covariance_matrix=`I`

In [0]:

class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, n_var, action_std):
        super(ActorCritic, self).__init__()
        # action mean range -1 to 1 #TODO how to change this to [0,1]
        self.actor =  nn.Sequential(
                nn.Linear(state_dim, n_var),
                nn.Tanh(),
                nn.Linear(n_var, n_var),
                nn.Tanh(),
                nn.Linear(n_var, action_dim),
                nn.Tanh()#todo change this to have something in the  [0, 1] interval ***************
                )
        # critic
        self.critic = nn.Sequential(  #TODO penser a supp ca since we have alredy a well defined reward
                nn.Linear(state_dim, n_var),
                nn.Tanh(),
                nn.Linear(n_var, n_var),
                nn.Tanh(),
                nn.Linear(n_var, 1)
                )
        self.action_var = torch.full((action_dim,), action_std*action_std).to(device) # define the variance of the action taken (gaussian model)
        self.action_upper_bound = 1.0 #kim
        self.action_lower_bound = 0.0 #kim
    def forward(self):
        raise NotImplementedError
    
    def act(self, state, memory):
        #print("act::state ",state)
        type(state)
        action_mean = self.actor(state)
        #print("action_mean: ", action_mean)
        dist = MultivariateNormal(action_mean, torch.diag(self.action_var).to(device)) # Gaussian distrib used by default by PPO
        # To get a bounded action space we use Beta distrib:
        # beta(4,4) approximate the gaussian well but still need to work on it ++ https://stats.stackexchange.com/questions/317729/is-the-gaussian-distribution-a-specific-case-of-the-beta-distribution
        #dist = Beta(torch.tensor([4.0,4.0]), torch.tensor([4.0,4.0])) # https://pytorch.org/docs/stable/distributions.html#beta
        # todo transform action_mean, torch.diag(self.action_var).to(device) --> beta param
        action = dist.sample()
        
        
        #print(action)
        action = action.tolist()
        action = [ (math.tanh(action[0])+1.0)/2.0 , (math.tanh(action[1])+1.0)/2.0  ] #https://pytorch.org/docs/stable/nn.html#tanh
        #print(action)
        #action+=1
        #print(action)
        #action/=2.0
        #print(action)
        action = torch.tensor(action)#this is making me use cpu and not gpu
        #print(action)
        
        
        
        
        action_logprob = dist.log_prob(action)
        memory.states.append(state)
        memory.actions.append(action)
        memory.logprobs.append(action_logprob)
        # ++ how to limit my action space to https://github.com/openai/baselines/issues/121
        
        # kim: i will implement this simple sol and hope it works: https://github.com/openai/baselines/issues/121#issuecomment-369688616
        # action = np.clip(action, self.action_space.low, self.action_space.high)
        # well since it s not a toch obj i will skip this sol XD --> let's use beta distrib (up) 
        # --> i don t know how to integrate the param of action_mean and action_var into the beta distrib --> go back to upper bound limit
        #todo use a wrapper func for this thing ++ https://hub.packtpub.com/openai-gym-environments-wrappers-and-monitors-tutorial/
        #action = torch.clamp(action, self.action_lower_bound, self.action_upper_bound, out=None) # https://pytorch.org/docs/master/torch.html?#torch.clamp
        #we get a lot of 0 and 1 even if it a good thing but it will not learn in an efficient way since 0 was the mean and we trucate in 0 XD
        
        
        return action.detach()
    
    def evaluate(self, state, action):
        action_mean = self.actor(state)
        dist = MultivariateNormal(torch.squeeze(action_mean), torch.diag(self.action_var))
        
        action_logprobs = dist.log_prob(torch.squeeze(action))
        dist_entropy = dist.entropy()
        state_value = self.critic(state)
        
        return action_logprobs, torch.squeeze(state_value), dist_entropy

class PPO:
    def __init__(self, state_dim, action_dim, n_latent_var, action_std, lr, betas, gamma, K_epochs, eps_clip):
        self.lr = lr
        self.betas = betas
        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs
        
        self.policy = ActorCritic(state_dim, action_dim, n_latent_var, action_std).to(device)
        self.optimizer = torch.optim.Adam(self.policy.parameters(),
                                              lr=lr, betas=betas)
        self.policy_old = ActorCritic(state_dim, action_dim, n_latent_var, action_std).to(device)
        
        self.MseLoss = nn.MSELoss()
    
    def select_action(self, state, memory):
        #print("select_action::state ", state)
        # state will not be reshaped (1, -1) since we have only one state -.-
        state = torch.FloatTensor(state).to(device) # todo probably have to change it to int
        #print("select_action:::state ", state)
        return self.policy_old.act(state, memory).cpu().data.numpy().flatten()
    
    def update(self, memory):
        # Monte Carlo estimate of rewards:
        rewards = []
        discounted_reward = 0
        for reward in reversed(memory.rewards):
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)
        
        # Normalizing the rewards:
        rewards = torch.tensor(rewards).to(device)
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-5)
        
        # convert list to tensor
        old_states = torch.stack(memory.states).to(device).detach()
        old_actions = torch.stack(memory.actions).to(device).detach()
        old_logprobs = torch.squeeze(torch.stack(memory.logprobs)).to(device).detach()
        
        # Optimize policy for K epochs:
        for _ in range(self.K_epochs):
            # Evaluating old actions and values :
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)
            # Finding the ratio (pi_theta / pi_theta__old):
            ratios = torch.exp(logprobs - old_logprobs.detach())

            # Finding Surrogate Loss:
            advantages = rewards - state_values.detach()
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages
            loss = -torch.min(surr1, surr2) + 0.5*self.MseLoss(state_values, rewards) - 0.01*dist_entropy
            
            # take gradient step
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()
            
        # Copy new weights into old policy:
        self.policy_old.load_state_dict(self.policy.state_dict())
        


In [0]:
#creating env of KDD  (extending gym) https://www.novatec-gmbh.de/en/blog/creating-a-gym-environment/
from gym import error, spaces, utils
from gym.utils import seeding
 
class KddEnv(gym.Env): 
    state_dim = 1 # we have only one space dim
    action_dim = 2 
    #metadata = {'render.modes': ['human']}   
    allRewards = []
    def __init__(self):
        print("--> init")
        self.envSeqDec = ChallengeSeqDecEnvironment() #Initialise a New Challenge Environment to post entire policy
    def step(self, action): # one action note hole episode
        type(action)
        type(action.squeeze().tolist() )
        s,r,d,_ = self.envSeqDec.evaluateAction(action.squeeze().tolist())
        print("--> action: ",action,"    s,r,d,_", s,"  ",r,"  ",d)
        self.allRewards.append(r)
        return [s],r,d,_
    def reset(self):
        print("--> reset")
        #self.envSeqDec.reset()
        self.envSeqDec = ChallengeSeqDecEnvironment()
        return [1]
    def render(self, mode='human', close=False):
        if len(self.allRewards) % 50 == 5 : 
          plt.plot(self.allRewards)
          #plt.xlim((0,120))
          plt.ylim((-100,150))
          plt.show()
          concat = np.add.reduceat(self.allRewards, np.arange(0, len(self.allRewards), 5))
          print(np.max(concat))

In [41]:
def main():
    ############## Hyperparameters ##############
    env_name = "KDD-v2"    #"LunarLanderContinuous-v2"
    render = True              # juste lel affichage bta3 el model (gym)
    solved_reward = 200    #200 # stop training if avg_reward > solved_reward
    log_interval = 5       #20  # print avg reward in the interval
    max_episodes = 2000#20      #50000# max training episodes
    max_timesteps = 5      #300 # max timesteps in one episode
    n_latent_var = 16 #8       #64    # number of variables in hidden layer
                                # https://stats.stackexchange.com/a/354476    :
                                # classic statistical advice to use the number of samples at least 10 times more than the number of parameters. This is vague, of course. If the problem is too noisy, you can demand 100 times more, or 1000 times more.
                                # we have a max of 100 cases (actions) or in worst case 20 cases (episode)
    update_timestep = 5    #4000# update policy every n timesteps ; nthon lezem max_timesteps
    action_std = 0.6 # 0.4       #0.6 # constant std for action distribution   # je sais pas quoi mettre mais je doit faire une hyperparm opt on it
                                # my action space is 2 time smaller than the original prob so i devided by sqrt(2)
    lr = 0.0025                 # learning rate : https://www.freecodecamp.org/news/how-to-pick-the-best-learning-rate-for-your-machine-learning-project-9c28865039a8/
                                # i will have to run a hyperparm opt on it even if it's the best value 
    betas = (0.9, 0.999)        # est ce que je devrai hyperOpt this or not ??? ++beta
    gamma = 0.99                # discount factor  # hyperOpt ++
    K_epochs = 5           #5   # update policy for K epochs
    eps_clip = 0.2              # clip parameter for PPO
    random_seed = None
    #############################################
    
    # creating environment
    env = KddEnv()
    if random_seed:
        print("Random Seed: {}".format(random_seed))
        torch.manual_seed(random_seed)
        env.seed(random_seed)
        np.random.seed(random_seed)
        
    memory = Memory()
    ppo = PPO(env.state_dim, env.action_dim, n_latent_var, action_std, lr, betas, gamma, K_epochs, eps_clip) # this beta have nothing todo with what i'm trying to add
    
    # logging variables
    running_reward = 0
    old_running_reward = 0
    avg_length = 0
    time_step = 0
    
    # training loop
    for i_episode in range(1, max_episodes+1):
        state = env.reset()
        for t in range(max_timesteps):
            time_step +=1
            print("time_step ",time_step )
            # Running policy_old:
            action = ppo.select_action(state, memory)
            state, reward, done, _ = env.step(action)
            # Saving reward:
            memory.rewards.append(reward)
            
            # update if its time
            if time_step % update_timestep == 0:
                ppo.update(memory)
                memory.clear_memory()
                time_step = 0
            running_reward += reward
            if render:
                env.render()
            if done:
                print("running_reward (tot) when done: ",running_reward - old_running_reward)
                old_running_reward = running_reward
                break
        
        avg_length += t
        
        # # stop training if avg_reward > solved_reward
        if running_reward > (log_interval*solved_reward):
            print("########## Solved! ##########")
            torch.save(ppo.policy.state_dict(), './PPO_Continuous_{}.pth'.format(env_name))
            break
        # logging
        if i_episode % log_interval == 0:
            avg_length = int(avg_length/log_interval)
            running_reward = int((running_reward/log_interval))
            
            print('Episode {} \t Avg length: {} \t Avg reward: {}'.format(i_episode, avg_length, running_reward))
            running_reward = 0
            avg_length = 0
            
if __name__ == '__main__':
    main()

Output hidden; open in https://colab.research.google.com to view.